# Desafio de Regressão - Tera

**Objetivo**: desenvolver um modelo de regressão linear (1) interpretável e (2) cheio de insights. Posteriormente, desenvolver um modelo de RandomForest focando na assertividade do modelo. 

**Suporte**:
- Orientações: https://www.youtube.com/watch?v=NC1P4AFTj54
- Exemplo: https://mkt.quintoandar.com.br/quanto-cobrar-de-aluguel/
- Conhecimento: https://christophm.github.io/interpretable-ml-book/

**Dados**: Boston House Pricing

**Pontos de importância:**
- O modelo de predição deverá ser usado em aplicativos web, então o modelo deveria ser leve.
- Desenvolvimento da solução ponta a ponta
- R2 mínimo de 0,85
- No máximo 6 variáveis, p-valor <= 5% e não colineares
- Peso igual ao erro da estimação muito acima e da estimação muito abaixo do valor real

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
#from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from tqdm import tqdm

#### Etapa 1 - Regressão Linear

In [8]:
df = pd.read_csv('dados/base.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    1460 non-null   int64  
 1   ClasseImovel          1460 non-null   int64  
 2   ClasseZona            1460 non-null   object 
 3   Fachada               1201 non-null   float64
 4   TamanhoLote           1460 non-null   int64  
 5   Rua                   1460 non-null   object 
 6   Beco                  91 non-null     object 
 7   FormaProp             1460 non-null   object 
 8   PlanoProp             1460 non-null   object 
 9   Servicos              1460 non-null   object 
 10  ConfigLote            1460 non-null   object 
 11  InclinacaoLote        1460 non-null   object 
 12  Bairro                1460 non-null   object 
 13  Estrada1              1460 non-null   object 
 14  Estrada2              1460 non-null   object 
 15  TipoHabitacao        

,Id,ClasseImovel,ClasseZona,Fachada,TamanhoLote,Rua,Beco,FormaProp,PlanoProp,Servicos,...,AreaPiscina,QualidadePiscina,QualidadeCerca,Outros,ValorOutros,MesVenda,AnoVenda,TipoVenda,CondicaoVenda,PrecoVenda
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### Baseline da Loucura

In [20]:
#atira primeiro e pergunta depois

#ver tipos de datatypes
print(f'Tipos de dados: {df.dtypes.unique()}')

#pegando apenas variáveis numéricas (não vou mentir, é apenas para ficar mais fácil)
#df_loucura = df.select_dtypes(include=['int64', 'float64']).dropna(how='any', axis=0)
df_loucura = df.select_dtypes(include=['int64', 'float64']).fillna(df.mean())

#quero standarizar para ter uma maneira fácil de comparar as variáveis
lista_colunas = df_loucura.columns
dados_standarizados = StandardScaler().fit_transform(df_loucura)
df_loucura_std = pd.DataFrame(dados_standarizados, columns=lista_colunas)

X = df_loucura_std[df_loucura.columns[:-1]]
y = df_loucura_std['PrecoVenda']

modelo = sm.OLS(y, X)
resultado = modelo.fit()
resultado.summary()

Tipos de dados: [dtype('int64') dtype('O') dtype('float64')]


C:\Users\Kenji\AppData\Local\Temp/ipykernel_12856/1296565544.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_loucura = df.select_dtypes(include=['int64', 'float64']).fillna(df.mean())


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             PrecoVenda   R-squared (uncentered):                   0.813
Model:                            OLS   Adj. R-squared (uncentered):              0.809
Method:                 Least Squares   F-statistic:                              177.2
Date:                Wed, 02 Feb 2022   Prob (F-statistic):                        0.00
Time:                        00:08:21   Log-Likelihood:                         -847.09
No. Observations:                1460   AIC:                                      1764.
Df Residuals:                    1425   BIC:                                      1949.
Df Model:                          35                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Id                      -0.0057      0.012     -0.493      0.622      -0.028       0.017
ClasseImovel            -0.0965      0.015     -6.546      0.000      -0.125      -0.068
Fachada                 -0.0157      0.014     -1.093      0.275      -0.044       0.012
TamanhoLote              0.0538      0.013      4.192      0.000       0.029       0.079
Qualidade                0.3012      0.021     14.556      0.000       0.261       0.342
Condicao                 0.0656      0.014      4.532      0.000       0.037       0.094
AnoConstrucao            0.1032      0.026      4.017      0.000       0.053       0.154
AnoReforma               0.0351      0.018      1.967      0.049       9e-05       0.070
AreaAlvenaria            0.0711      0.014      5.253      0.000       0.045       0.098
AreaAcabPorao1           0.0545      0.013      4.274      0.000       0.029       0.080
AreaAcabPorao2          -0.0030      0.012     -0.253      0.800      -0.026       0.020
AreaInacabPorao         -0.0022      0.011     -0.199      0.842      -0.024       0.019
AreaPorao                0.0534      0.016      3.391      0.001       0.022       0.084
AreaTerreo               0.1072      0.019      5.732      0.000       0.070       0.144
Area2Andar               0.1205      0.016      7.389      0.000       0.088       0.152
BaixaQualiAreaAcab      -0.0012      0.012     -0.101      0.919      -0.025       0.023
AreaConstruida           0.1788      0.017     10.512      0.000       0.145       0.212
BanheiroPorao            0.0612      0.017      3.588      0.000       0.028       0.095
LavaboPorao              0.0060      0.012      0.490      0.624      -0.018       0.030
Banheiro                 0.0240      0.020      1.220      0.223      -0.015       0.063
Lavabo                  -0.0119      0.017     -0.703      0.482      -0.045       0.021
BedroomAbvGr            -0.1035      0.017     -5.923      0.000      -0.138      -0.069
KitchenAbvGr            -0.0342      0.014     -2.364      0.018      -0.063      -0.006
TotalQuartos             0.1040      0.025      4.107      0.000       0.054       0.154
Lareira                  0.0320      0.014      2.222      0.026       0.004       0.060
AnoGaragem               0.0369      0.021      1.755      0.079      -0.004       0.078
CarrosGaragem            0.1059      0.027      3.917      0.000       0.053       0.159
AreaGaragem             -0.0113      0.027     -0.422      0.673      -0.064       0.041
AreaDeck                 0.0377      0.013      2.977      0.003       0.013       0.062
AreaVarandaAberta       -0.0024      0.013     -0.192      0.848      -0.027       0.022
AreaVarandaFechada       0.0091      0.013     

In [21]:
#considerando apenas o efeito e o valor t
df_loucura2 = pd.DataFrame([resultado.params, resultado.tvalues], index=['params','tvalues']).T

#filtrando apenas variáveis significantes
df_loucura2 = df_loucura2.loc[abs(df_loucura2['tvalues'] >= 3)]

#a partir dessa lista, faremos uma séries de loops para identificar a combinação das 6 variáveis que apresentem:
# 1) p-valor <= 5%
# 2) params relevante
# 3) VIF <= 10

lista_var_total_loucura = abs(df_loucura2['params']).sort_values(ascending=False).index.tolist()
print(f'Lista das variáveis mais promissoras considerando valor t:\n {lista_var_total_loucura}')

#ajuste na mão
lista_var_total_loucura.remove('BanheiroPorao')

def loop_eterno(lista_var_total_loucura, df):

    for i in range(len(lista_var_total_loucura)):
        var1 = lista_var_total_loucura[i]

        for j in range(len(lista_var_total_loucura)):
            var2 = lista_var_total_loucura[j]

            for k in range(len(lista_var_total_loucura)):
                var3 = lista_var_total_loucura[k]

                for l in range(len(lista_var_total_loucura)):
                    var4 = lista_var_total_loucura[l]

                    for m in range(len(lista_var_total_loucura)):
                        var5 = lista_var_total_loucura[m]

                        for n in range(len(lista_var_total_loucura)):
                            var6 = lista_var_total_loucura[n]

                            lista_6var = [var1, var2, var3, var4, var5, var6]
                            
                            if len(set(lista_6var)) == 6:
                                X_temp = df[[var1, var2, var3, var4, var5, var6]]
                                resultado_vif = [vif(X_temp.values, i) for i in range(len(X_temp.columns))]
                                
                                if np.all(np.array(resultado_vif) <= 10):
                                    print(f'Sucesso: {X_temp.columns.tolist()}')
                                    print(f'VIF: {resultado_vif}')
                                    return lista_6var

lista_6var = loop_eterno(lista_var_total_loucura, df_loucura)

X = df_loucura[lista_6var]
y = df_loucura['PrecoVenda']

modelo = sm.OLS(y, X)
resultado = modelo.fit()
resultado.summary()

Lista das variáveis mais promissoras considerando valor t:
 ['Qualidade', 'AreaConstruida', 'Area2Andar', 'AreaTerreo', 'CarrosGaragem', 'TotalQuartos', 'AnoConstrucao', 'AreaAlvenaria', 'Condicao', 'BanheiroPorao', 'AreaAcabPorao1', 'TamanhoLote', 'AreaPorao', 'AreaAlpendre']
Sucesso: ['Qualidade', 'Area2Andar', 'AreaAlvenaria', 'AreaAcabPorao1', 'TamanhoLote', 'AreaAlpendre']
VIF: [4.1569951592021965, 1.89633582587226, 1.5497031432679, 2.3657230818652955, 2.1862712899016237, 1.0826911251414437]


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             PrecoVenda   R-squared (uncentered):                   0.947
Model:                            OLS   Adj. R-squared (uncentered):              0.946
Method:                 Least Squares   F-statistic:                              4301.
Date:                Wed, 02 Feb 2022   Prob (F-statistic):                        0.00
Time:                        00:08:51   Log-Likelihood:                         -17735.
No. Observations:                1460   AIC:                                  3.548e+04
Df Residuals:                    1454   BIC:                                  3.551e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Qualidade       2.334e+04    390.118     59.825      0.000    2.26e+04    2.41e+04
Area2Andar        26.9728      2.956      9.126      0.000      21.175      32.771
AreaAlvenaria     85.1042      7.156     11.892      0.000      71.067      99.142
AreaAcabPorao1    32.9713      2.893     11.396      0.000      27.296      38.647
TamanhoLote        0.8161      0.122      6.687      0.000       0.577       1.056
AreaAlpendre      54.9825     21.566      2.550      0.011      12.679      97.286
==============================================================================
Omnibus:                      532.182   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11010.142
Skew:                           1.181   Prob(JB):                         0.00
Kurtosis:                      16.244   Cond. No.                     4.73e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 4.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Beleza, o modelo de regressão baseline é formado por 6 variáveis: 
- _Qualidade_: material geral e qualidade de acabamento (1 a 10);
- _Area2Andar_: área do segundo andar em pés quadrado;
- _AreaAlvenaria_: área de maneira em pés quadrado;
- AreaAcabPorao1: área do porão em pés quadrado;
- _TamanhoLote_: tamanho do terreno em pés quadrado;
- _AreaAlpendre_: área coberta de varanda em pés quadrado;

obs: 1 feet = 0,09m

## Etapa 2 - Random Forest

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rfr = RandomForestRegressor().fit(X_train, y_train)
print(rfr.score(X_test, y_test))

y_pred = rfr.predict(X_test)

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_test, y_pred, squared=False))

0.8136294824196879
Mean Absolute Error (MAE): 24679.451643835615
Mean Squared Error (MSE): 1429522665.2432966
Root Mean Squared Error (RMSE): 37809.02888521863


In [37]:
y.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: PrecoVenda, dtype: float64